# Accuracy-Efficiency Table

Calculate Compute

Normalize job times by compute used.

In [98]:
# Packages
import pandas as pd
import os
import glob
import dataframe_image as dfi # NOTE: YOU MUST HAVE GOOGLE CHROME INSTALLED FOR THIS TO WORK CORRECTLY

## Job History Data

In [99]:
# jobs and cleaned up names for them
jobs_and_clean_names = {
                'Fit_Logistic_Regression_Classifier_All_Data': 'Logistic Regression-All Features',
                'Fit_Logistic_Regression_Classifier_All_Features_PCA' : 'Logistic Regression-All Features PCA',
                'Fit_Logistic_Regression_Classifier_Individual_Features_PCA' : 'Logistic Regression-Individual Features PCA',
                #'Fit_SVM_Classifier_All_Data': 'SVM-All Features',
                'Fit_SVM_Classifier_All_Features_PCA': 'SVM-All Features PCA',
                'Fit_SVM_Classifier_Individual_Features_PCA': 'SVM-Individual Features PCA',
                'fit_XGBoost_All_data': 'XGBoost-All Features',
                'fit_XGBoost_All_features_PCA': 'XGBoost-All Features PCA',
                'fit_XGBoost_Individual_Features_PCA': 'XGBoost-Individual Features PCA',
}

In [100]:
# Read pipe-delimited file '../../Data/Compute/job_history.txt' into a DataFrame
job_history = pd.read_csv('../../Data/Compute/job_history.txt', delimiter='|')
job_history

,JobID,JobName,Partition,State,Start,End,AllocNodes,AllocCPUS,NodeList
0,385319,jupyterhub,standalone,CANCELLED by 3581,2024-03-09T14:21:23,2024-03-09T23:02:28,1,1,arwen
1,385319.batch,batch,NaN,CANCELLED,2024-03-09T14:21:23,2024-03-09T23:03:00,1,1,arwen
2,385319.extern,extern,NaN,COMPLETED,2024-03-09T14:21:23,2024-03-09T23:02:28,1,1,arwen
3,385387,transcript_ner,jsteinhardt,FAILED,2024-03-09T21:07:13,2024-03-09T21:07:19,1,1,saruman
4,385387.batch,batch,NaN,FAILED,2024-03-09T21:07:13,2024-03-09T21:07:19,1,1,saruman
...,...,...,...,...,...,...,...,...,...
730,439672.extern,extern,NaN,COMPLETED,2024-04-09T21:25:33,2024-04-09T21:26:16,1,1,smokyquartz
731,439672.0,bash,NaN,COMPLETED,2024-04-09T21:25:33,2024-04-09T21:26:16,1,1,smokyquartz
732,439673,bash,jsteinhardt,COMPLETED,2024-04-09T21:26:33,2024-04-09T21:27:09,1,1,rainbowquartz
733,439673.extern,extern,NaN,COMPLETED,2024-04-09T21:26:33,2024-04-09T21:27:09,1,1,rainbowquartz


In [101]:
# Selected jobs - Filter job_history DataFrame to only include jobs in jobs_and_clean_names keys (job names)
# require State == 'COMPLETED'
# select row of maximum End time for each job
selected_jobs = (job_history[job_history['JobName'].isin(jobs_and_clean_names.keys())]
                                                   .query('State == "COMPLETED"')
                                                   .groupby('JobName')
                                                   .apply(lambda x: x.loc[x['End'].idxmax()])
                                                   .reset_index(drop=True)
                                                   )
# Get time elapsed by differencing Start and End times
selected_jobs['TimeElapsed'] = pd.to_datetime(selected_jobs['End']) - pd.to_datetime(selected_jobs['Start'])
selected_jobs

,JobID,JobName,Partition,State,Start,End,AllocNodes,AllocCPUS,NodeList,TimeElapsed
0,434001,Fit_Logistic_Regression_Classifier_All_Data,jsteinhardt,COMPLETED,2024-04-07T15:19:21,2024-04-08T16:23:21,1,16,shadowfax,1 days 01:04:00
1,434003,Fit_Logistic_Regression_Classifier_All_Feature...,jsteinhardt,COMPLETED,2024-04-07T15:19:38,2024-04-07T19:27:29,1,16,smokyquartz,0 days 04:07:51
2,434002,Fit_Logistic_Regression_Classifier_Individual_...,jsteinhardt,COMPLETED,2024-04-07T15:19:30,2024-04-07T22:24:49,1,16,shadowfax,0 days 07:05:19
3,435811,Fit_SVM_Classifier_All_Features_PCA,jsteinhardt,COMPLETED,2024-04-07T23:26:10,2024-04-08T02:21:37,1,16,sunstone,0 days 02:55:27
4,435809,Fit_SVM_Classifier_Individual_Features_PCA,jsteinhardt,COMPLETED,2024-04-07T23:25:50,2024-04-08T07:05:06,1,16,sunstone,0 days 07:39:16
5,434006,fit_XGBoost_All_data,jsteinhardt,COMPLETED,2024-04-07T15:20:30,2024-04-07T19:15:20,1,16,smokyquartz,0 days 03:54:50
6,434005,fit_XGBoost_All_features_PCA,epurdom,COMPLETED,2024-04-07T15:20:20,2024-04-07T15:55:20,1,16,frodo,0 days 00:35:00
7,434004,fit_XGBoost_Individual_Features_PCA,epurdom,COMPLETED,2024-04-07T15:20:11,2024-04-07T16:02:43,1,16,frodo,0 days 00:42:32


## CPU data

In [102]:
# Load '../../Data/Compute/scf_cpus.xlsx'
scf_cpus = pd.read_excel('../../Data/Compute/scf_cpus.xlsx')
scf_cpus

,partition,node,CPU model,passmark score
0,jsteinhardt,shadowfax,Intel Xeon Silver 4214 CPU @ 2.20GHz,16190
1,jsteinhardt,balrog,Intel Xeon Gold 5220R CPU @ 2.20GHz,33370
2,jsteinhardt,saruman,Intel Xeon Gold 5320 CPU @ 2.20GHz,37558
3,jsteinhardt,sunstone,Intel Xeon Gold 5218 CPU @ 2.30GHz,21986
4,epurdom,frodo,AMD EPYC 7543 32-Core Processor,60132
5,jsteinhardt,smokyquartz,Intel Xeon Gold 5218 CPU @ 2.30GHz,21986
6,jsteinhardt,rainbowquartz,Intel Xeon Gold 5218 CPU @ 2.30GHz,21986


## Join on NodeList and node

In [103]:
job_cpu = pd.merge(selected_jobs, scf_cpus, left_on = 'NodeList', right_on = 'node', how = 'left')
job_cpu

,JobID,JobName,Partition,State,Start,End,AllocNodes,AllocCPUS,NodeList,TimeElapsed,partition,node,CPU model,passmark score
0,434001,Fit_Logistic_Regression_Classifier_All_Data,jsteinhardt,COMPLETED,2024-04-07T15:19:21,2024-04-08T16:23:21,1,16,shadowfax,1 days 01:04:00,jsteinhardt,shadowfax,Intel Xeon Silver 4214 CPU @ 2.20GHz,16190
1,434003,Fit_Logistic_Regression_Classifier_All_Feature...,jsteinhardt,COMPLETED,2024-04-07T15:19:38,2024-04-07T19:27:29,1,16,smokyquartz,0 days 04:07:51,jsteinhardt,smokyquartz,Intel Xeon Gold 5218 CPU @ 2.30GHz,21986
2,434002,Fit_Logistic_Regression_Classifier_Individual_...,jsteinhardt,COMPLETED,2024-04-07T15:19:30,2024-04-07T22:24:49,1,16,shadowfax,0 days 07:05:19,jsteinhardt,shadowfax,Intel Xeon Silver 4214 CPU @ 2.20GHz,16190
3,435811,Fit_SVM_Classifier_All_Features_PCA,jsteinhardt,COMPLETED,2024-04-07T23:26:10,2024-04-08T02:21:37,1,16,sunstone,0 days 02:55:27,jsteinhardt,sunstone,Intel Xeon Gold 5218 CPU @ 2.30GHz,21986
4,435809,Fit_SVM_Classifier_Individual_Features_PCA,jsteinhardt,COMPLETED,2024-04-07T23:25:50,2024-04-08T07:05:06,1,16,sunstone,0 days 07:39:16,jsteinhardt,sunstone,Intel Xeon Gold 5218 CPU @ 2.30GHz,21986
5,434006,fit_XGBoost_All_data,jsteinhardt,COMPLETED,2024-04-07T15:20:30,2024-04-07T19:15:20,1,16,smokyquartz,0 days 03:54:50,jsteinhardt,smokyquartz,Intel Xeon Gold 5218 CPU @ 2.30GHz,21986
6,434005,fit_XGBoost_All_features_PCA,epurdom,COMPLETED,2024-04-07T15:20:20,2024-04-07T15:55:20,1,16,frodo,0 days 00:35:00,epurdom,frodo,AMD EPYC 7543 32-Core Processor,60132
7,434004,fit_XGBoost_Individual_Features_PCA,epurdom,COMPLETED,2024-04-07T15:20:11,2024-04-07T16:02:43,1,16,frodo,0 days 00:42:32,epurdom,frodo,AMD EPYC 7543 32-Core Processor,60132


## Compute units - compute time times num cpus times passmark score

In [104]:
# compute units - TimeElapsed * CPUs * passmark score
job_cpu['Units'] = job_cpu['TimeElapsed'].dt.total_seconds() * job_cpu['AllocCPUS'] * job_cpu['passmark score']
job_cpu

,JobID,JobName,Partition,State,Start,End,AllocNodes,AllocCPUS,NodeList,TimeElapsed,partition,node,CPU model,passmark score,Units
0,434001,Fit_Logistic_Regression_Classifier_All_Data,jsteinhardt,COMPLETED,2024-04-07T15:19:21,2024-04-08T16:23:21,1,16,shadowfax,1 days 01:04:00,jsteinhardt,shadowfax,Intel Xeon Silver 4214 CPU @ 2.20GHz,16190,2.337577e+10
1,434003,Fit_Logistic_Regression_Classifier_All_Feature...,jsteinhardt,COMPLETED,2024-04-07T15:19:38,2024-04-07T19:27:29,1,16,smokyquartz,0 days 04:07:51,jsteinhardt,smokyquartz,Intel Xeon Gold 5218 CPU @ 2.30GHz,21986,5.231261e+09
2,434002,Fit_Logistic_Regression_Classifier_Individual_...,jsteinhardt,COMPLETED,2024-04-07T15:19:30,2024-04-07T22:24:49,1,16,shadowfax,0 days 07:05:19,jsteinhardt,shadowfax,Intel Xeon Silver 4214 CPU @ 2.20GHz,16190,6.610442e+09
3,435811,Fit_SVM_Classifier_All_Features_PCA,jsteinhardt,COMPLETED,2024-04-07T23:26:10,2024-04-08T02:21:37,1,16,sunstone,0 days 02:55:27,jsteinhardt,sunstone,Intel Xeon Gold 5218 CPU @ 2.30GHz,21986,3.703146e+09
4,435809,Fit_SVM_Classifier_Individual_Features_PCA,jsteinhardt,COMPLETED,2024-04-07T23:25:50,2024-04-08T07:05:06,1,16,sunstone,0 days 07:39:16,jsteinhardt,sunstone,Intel Xeon Gold 5218 CPU @ 2.30GHz,21986,9.693539e+09
5,434006,fit_XGBoost_All_data,jsteinhardt,COMPLETED,2024-04-07T15:20:30,2024-04-07T19:15:20,1,16,smokyquartz,0 days 03:54:50,jsteinhardt,smokyquartz,Intel Xeon Gold 5218 CPU @ 2.30GHz,21986,4.956524e+09
6,434005,fit_XGBoost_All_features_PCA,epurdom,COMPLETED,2024-04-07T15:20:20,2024-04-07T15:55:20,1,16,frodo,0 days 00:35:00,epurdom,frodo,AMD EPYC 7543 32-Core Processor,60132,2.020435e+09
7,434004,fit_XGBoost_Individual_Features_PCA,epurdom,COMPLETED,2024-04-07T15:20:11,2024-04-07T16:02:43,1,16,frodo,0 days 00:42:32,epurdom,frodo,AMD EPYC 7543 32-Core Processor,60132,2.455310e+09


## Get compute time, roughly, if the job was run on laptop

11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80 GHz

Passmark: 10,293

In [105]:
# Add estimated laptop time as Units divided by 10,293
job_cpu['LaptopTime'] = job_cpu['Units'] / 10293
# Convert to minutes by dividing by 60
job_cpu['LaptopTime minutes'] = job_cpu['LaptopTime'] / 60
# Convert to hours by dividing by 60
job_cpu['LaptopTime hours'] = job_cpu['LaptopTime minutes'] / 60
job_cpu

,JobID,JobName,Partition,State,Start,End,AllocNodes,AllocCPUS,NodeList,TimeElapsed,partition,node,CPU model,passmark score,Units,LaptopTime,LaptopTime minutes,LaptopTime hours
0,434001,Fit_Logistic_Regression_Classifier_All_Data,jsteinhardt,COMPLETED,2024-04-07T15:19:21,2024-04-08T16:23:21,1,16,shadowfax,1 days 01:04:00,jsteinhardt,shadowfax,Intel Xeon Silver 4214 CPU @ 2.20GHz,16190,2.337577e+10,2.271036e+06,37850.593607,630.843227
1,434003,Fit_Logistic_Regression_Classifier_All_Feature...,jsteinhardt,COMPLETED,2024-04-07T15:19:38,2024-04-07T19:27:29,1,16,smokyquartz,0 days 04:07:51,jsteinhardt,smokyquartz,Intel Xeon Gold 5218 CPU @ 2.30GHz,21986,5.231261e+09,5.082348e+05,8470.580161,141.176336
2,434002,Fit_Logistic_Regression_Classifier_Individual_...,jsteinhardt,COMPLETED,2024-04-07T15:19:30,2024-04-07T22:24:49,1,16,shadowfax,0 days 07:05:19,jsteinhardt,shadowfax,Intel Xeon Silver 4214 CPU @ 2.20GHz,16190,6.610442e+09,6.422269e+05,10703.782117,178.396369
3,435811,Fit_SVM_Classifier_All_Features_PCA,jsteinhardt,COMPLETED,2024-04-07T23:26:10,2024-04-08T02:21:37,1,16,sunstone,0 days 02:55:27,jsteinhardt,sunstone,Intel Xeon Gold 5218 CPU @ 2.30GHz,21986,3.703146e+09,3.597732e+05,5996.220655,99.937011
4,435809,Fit_SVM_Classifier_Individual_Features_PCA,jsteinhardt,COMPLETED,2024-04-07T23:25:50,2024-04-08T07:05:06,1,16,sunstone,0 days 07:39:16,jsteinhardt,sunstone,Intel Xeon Gold 5218 CPU @ 2.30GHz,21986,9.693539e+09,9.417604e+05,15696.006114,261.600102
5,434006,fit_XGBoost_All_data,jsteinhardt,COMPLETED,2024-04-07T15:20:30,2024-04-07T19:15:20,1,16,smokyquartz,0 days 03:54:50,jsteinhardt,smokyquartz,Intel Xeon Gold 5218 CPU @ 2.30GHz,21986,4.956524e+09,4.815432e+05,8025.719486,133.761991
6,434005,fit_XGBoost_All_features_PCA,epurdom,COMPLETED,2024-04-07T15:20:20,2024-04-07T15:55:20,1,16,frodo,0 days 00:35:00,epurdom,frodo,AMD EPYC 7543 32-Core Processor,60132,2.020435e+09,1.962922e+05,3271.535995,54.525600
7,434004,fit_XGBoost_Individual_Features_PCA,epurdom,COMPLETED,2024-04-07T15:20:11,2024-04-07T16:02:43,1,16,frodo,0 days 00:42:32,epurdom,frodo,AMD EPYC 7543 32-Core Processor,60132,2.455310e+09,2.385417e+05,3975.695171,66.261586


## Load Inference Time

In [106]:
# Load get each Excel file in subdirectories of '../../Output/Classifier Inference' into a DataFrame
# and concatenate them into a single DataFrame

# Get all Excel files in subdirectories of '../../Output/Classifier Inference'
all_files = glob.glob('../../Output/Classifier Inference/**/*.xlsx', recursive=True)
all_files

# Read each Excel file into a DataFrame and concatenate them into a single DataFrame
inference_time_df = pd.concat([pd.read_excel(f) for f in all_files], ignore_index=True)
# Add column of filenames
inference_time_df['Filename Full'] = [f for f in all_files]
# Change '\\' to '/' in 'Filename Full'
inference_time_df['Filename Full'] = inference_time_df['Filename Full'].str.replace('\\', '/')
# Model - second to last part of 'Filename Full' split by '/'
inference_time_df['Model'] = inference_time_df['Filename Full'].str.split('/').str[-2]
# Features - last part of 'Filename Full' split by '/'
inference_time_df['Features'] = inference_time_df['Filename Full'].str.split('/').str[-1]
# Strip " Prediction Statistics.xlsx" from 'Features'
inference_time_df['Features'] = inference_time_df['Features'].str.replace(' Prediction Statistics.xlsx', '')
# Recode 'Features'
mapping = {'All_Features_PCA': 'All Features PCA', 'Individual_Features_PCA': 'Individual Features PCA', 'All_Data': 'All Features'}
inference_time_df['Features'] = inference_time_df['Features'].replace(mapping)
# Limit columns to 'Model', 'Features', 'Inference Time (Seconds)' (runtime_minutes edited)
inference_time_df['Inference Time (Seconds)'] = inference_time_df['runtime_minutes'] * 60
inference_time_df = inference_time_df[['Model', 'Features', 'Inference Time (Seconds)']]
inference_time_df

,Model,Features,Inference Time (Seconds)
0,Logistic Regression,All Features,0.018357
1,Logistic Regression,All Features PCA,0.041061
2,Logistic Regression,Individual Features PCA,0.001303
3,SVM,All Features,0.000549
4,SVM,All Features PCA,6.114565
5,SVM,Individual Features PCA,6.531546
6,XGBoost,All Features,0.062092
7,XGBoost,All Features PCA,0.023631
8,XGBoost,Individual Features PCA,0.019087


## Accuracy - Efficiency Table

In [107]:
# Load model performance table
# '../../Output/Classifier Evaluation/Model Performance Table.xlsx'
model_performance = pd.read_excel('../../Output/Classifier Evaluation/Model Performance Table.xlsx')
model_performance

,Model,Features,Tuned Hyperparameters,Train Accuracy,Mean Cross-Validated Accuracy,Test Accuracy
0,Logistic Regression,All Features,"C: 0.1, class_weight: balanced, l1_ratio: 1.0,...",0.984175,0.915708,0.910678
1,Logistic Regression,All Features PCA,"C: 0.1, class_weight: None, l1_ratio: 0.5, mul...",0.992171,0.904881,0.890531
2,Logistic Regression,Individual Features PCA,"C: 0.01, class_weight: balanced, l1_ratio: 0.2...",0.958021,0.915876,0.909335
3,SVM,All Features PCA,"C: 0.1, degree: 2, gamma: 0.1, kernel: poly",1.000000,0.503248,0.574211
4,SVM,Individual Features PCA,"C: 0.1, degree: 2, gamma: 0.1, kernel: poly",1.000000,0.579708,0.631296
5,XGBoost,All Features,"colsample_bytree: 1, learning_rate: 0.3, max_d...",1.000000,0.909380,0.901276
6,XGBoost,All Features PCA,"colsample_bytree: 1, learning_rate: 0.3, max_d...",1.000000,0.855573,0.862995
7,XGBoost,Individual Features PCA,"colsample_bytree: 0.8, learning_rate: 0.3, max...",1.000000,0.888056,0.881128


In [108]:
# Cleaned up table of cpu usage
# Limit job_cpu to only include columns 'JobName', 'LaptopTime hours'
job_cpu_clean = job_cpu[['JobName', 'LaptopTime hours']]
# Create column jobname_clean as the cleaned up job name from jobs_and_clean_names
job_cpu_clean['jobname_clean'] = job_cpu_clean['JobName'].map(jobs_and_clean_names)
# Split jobname_clean into two columns 'Model' and 'Features' on '-'
job_cpu_clean[['Model', 'Features']] = job_cpu_clean['jobname_clean'].str.split('-', expand=True)
# Drop 'JobName' and 'jobname_clean' columns
job_cpu_clean.drop(columns=['JobName', 'jobname_clean'], inplace=True)
job_cpu_clean

C:\Users\ijyli\AppData\Local\Temp\ipykernel_4804\3029528637.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  job_cpu_clean['jobname_clean'] = job_cpu_clean['JobName'].map(jobs_and_clean_names)
C:\Users\ijyli\AppData\Local\Temp\ipykernel_4804\3029528637.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  job_cpu_clean[['Model', 'Features']] = job_cpu_clean['jobname_clean'].str.split('-', expand=True)
C:\Users\ijyli\AppData\Local\Temp\ipykernel_4804\3029528637.py:7: SettingWithCopyWarning: 
A value is tryi

,LaptopTime hours,Model,Features
0,630.843227,Logistic Regression,All Features
1,141.176336,Logistic Regression,All Features PCA
2,178.396369,Logistic Regression,Individual Features PCA
3,99.937011,SVM,All Features PCA
4,261.600102,SVM,Individual Features PCA
5,133.761991,XGBoost,All Features
6,54.525600,XGBoost,All Features PCA
7,66.261586,XGBoost,Individual Features PCA


In [109]:
# Accuracy - Efficiency table - join on 'Model' and 'Features'
eff_acc_table = pd.merge(model_performance, job_cpu_clean, on=['Model', 'Features'], how='left').merge(inference_time_df, on=['Model', 'Features'], how='left')
# Keep columns Model, Features, LaptopTime hours, Test Accuracy
eff_acc_table = eff_acc_table[['Model', 'Features', 'LaptopTime hours', 'Inference Time (Seconds)', 'Test Accuracy']]
# Rename LaptopTime hours to Training Time (Hours)
eff_acc_table.rename(columns={'LaptopTime hours': 'Training Time (Hours)'}, inplace=True)
eff_acc_table

,Model,Features,Training Time (Hours),Inference Time (Seconds),Test Accuracy
0,Logistic Regression,All Features,630.843227,0.018357,0.910678
1,Logistic Regression,All Features PCA,141.176336,0.041061,0.890531
2,Logistic Regression,Individual Features PCA,178.396369,0.001303,0.909335
3,SVM,All Features PCA,99.937011,6.114565,0.574211
4,SVM,Individual Features PCA,261.600102,6.531546,0.631296
5,XGBoost,All Features,133.761991,0.062092,0.901276
6,XGBoost,All Features PCA,54.525600,0.023631,0.862995
7,XGBoost,Individual Features PCA,66.261586,0.019087,0.881128


In [110]:
# Formatting and export
eff_acc_table = (eff_acc_table.style.format(thousands=",", decimal=".")
                                    .set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
                                    .set_properties(**{'text-align': 'center'})
                                    .hide(axis="index"))

# Save to PNG
dfi.export(eff_acc_table, '../../Output/Classifier Evaluation/Efficiency-Accuracy Table.png')

# Save to Excel
eff_acc_table.to_excel('../../Output/Classifier Evaluation/Efficiency-Accuracy Table.xlsx', index=False)

eff_acc_table

Model,Features,Training Time (Hours),Inference Time (Seconds),Test Accuracy
Logistic Regression,All Features,630.843227,0.018357,0.910678
Logistic Regression,All Features PCA,141.176336,0.041061,0.890531
Logistic Regression,Individual Features PCA,178.396369,0.001303,0.909335
SVM,All Features PCA,99.937011,6.114565,0.574211
SVM,Individual Features PCA,261.600102,6.531546,0.631296
XGBoost,All Features,133.761991,0.062092,0.901276
XGBoost,All Features PCA,54.525600,0.023631,0.862995
XGBoost,Individual Features PCA,66.261586,0.019087,0.881128
